In [2]:
import wptools
import json
from tqdm import tqdm
import re
import requests as r
from collections import defaultdict

In [3]:
class Translation_Api():
    # https://docs.microsoft.com/en-us/azure/cognitive-services/translator/reference/v3-0-reference
    def __init__(self):
        self.headers = {
                "Ocp-Apim-Subscription-Key":"e669322a133044489e6dc4e9cde3edee",
                "Content-Type":"application/json",
                "Ocp-Apim-Subscription-Region":"centralus"
            }
        self.transliterate_url = "https://api.cognitive.microsofttranslator.com/transliterate?api-version=3.0&language=hi&fromScript=Latn&toScript=Deva"
        self.translate_url = "https://api.cognitive.microsofttranslator.com/translate?api-version=3.0&to=hi&from=en&toScript=Deva"
    
    def get_translation(self,data):
        if "{{" in data : return data
        data = data.split("<ref>")[0]
        if data == "" : return data
        data = [{"Text":data}]
        res = r.post( self.translate_url , json = data , headers = self.headers).text
        res = json.loads(res)
        res = [ ret['translations'][0]['text'] for ret in res ]
        return res[0].replace("[[","[").replace("]]","]").replace("[","[[").replace("]","]]")
        
    def get_transliteration(self,data):
        if "{{" in data : return data
        data = data.split("<ref>")[0]
        if data == "" : return data
        data = [{"Text":data}]
        res = r.post( self.transliterate_url , json = data , headers = self.headers).text
        res = json.loads(res)
        res = [ret['text'] for ret in res]
        return res[0].replace("[[","[").replace("]]","]").replace("[","[[").replace("]","]]")

In [4]:
translator = Translation_Api()

In [5]:
data = "Sample Text"
translator.get_translation(data)

'नमूना पाठ'

In [6]:
data = "Sample Text"
translator.get_transliteration(data)

'सैंपल टेक्ट'

In [37]:
page = wptools.page('Navassa Island').get_parse()

en.wikipedia.org (parse) Navassa Island
Navassa Island (en) data
{
  infobox: <dict(11)> name, image_name, image_caption, pushpin_map...
  iwlinks: <list(5)> https://commons.wikimedia.org/wiki/Special:Se...
  pageid: 21312
  parsetree: <str(38808)> <root><template><title>Short description...
  requests: <list(1)> parse
  title: Navassa Island
  wikibase: Q25359
  wikidata_url: https://www.wikidata.org/wiki/Q25359
  wikitext: <str(27085)> {{Short description|Island in the Caribbe...
}


In [52]:
def update(infobox , translator):
    infobox = defaultdict(str , infobox)
    updated_infobox = {}
    updated_infobox['name'] = translator.get_transliteration(infobox['name'])
    updated_infobox['native_name'] = infobox['native_name']
    updated_infobox['category'] = translator.get_translation(infobox['settlement_type'])
    
    updated_infobox['image'] = infobox['image_skyline']
    if updated_infobox['image'] == '' : 
        updated_infobox['image'] = infobox['image']
    updated_infobox['image_caption'] = translator.get_translation(infobox['image_caption'])
    updated_infobox['flag'] = infobox['flag']
    updated_infobox['map'] = infobox['map']
    updated_infobox['map_caption'] = translator.get_translation(infobox['map_caption'])
    updated_infobox['motto'] = translator.get_transliteration(infobox['motto'])
    updated_infobox['timezone'] = infobox['timezone']
    
    updated_infobox['country'] = translator.get_translation(infobox['country'])
    updated_infobox['state'] = translator.get_translation(infobox['state'])
    updated_infobox['region'] = translator.get_translation(infobox['region'])
    updated_infobox['district'] = translator.get_translation(infobox['district'])
    updated_infobox['municipality'] = translator.get_translation(infobox['municipality'])
    updated_infobox['location'] = translator.get_translation(infobox['location'])
    updated_infobox['area'] = infobox['area_km2']
    updated_infobox['length'] = infobox['length_km']
    updated_infobox['width'] = infobox['width_km']
    
    
    updated_infobox['population'] = translator.get_translation(infobox['population'])
    updated_infobox['parent'] = translator.get_translation(infobox['parent'])
    updated_infobox['range'] = translator.get_translation(infobox['range'])
    updated_infobox['population'] = translator.get_translation(infobox['population'])
    updated_infobox['elevation'] = infobox['elevation_m']
    
    updated_infobox['animal'] = translator.get_translation(infobox['animal'])
    updated_infobox['plant'] = translator.get_translation(infobox['plant'])
    updated_infobox['geology'] = translator.get_translation(infobox['geology'])

    updated_infobox = { key : val for key , val in updated_infobox.items() if val!=''}
    cur_len = len(updated_infobox)
    if len(updated_infobox) < 15:
        for key , val in infobox.items():
            if key in updated_infobox : continue
            updated_infobox[key] = translator.get_transliteration(val)
            if len(updated_infobox) == 15: break
    updated_infobox = { key : val for key , val in updated_infobox.items() if val!=''}
    return updated_infobox

In [53]:
new_infobox = update(page.data['infobox'] , translator)

In [54]:
def change_format(infobox):
    infobox = [ ("|" + key + " = " +  val) for key , val in infobox.items()]
    print("{{Geobox")
    print("\n".join(infobox))
    print("}}")

In [55]:
new_infobox_string = change_format(new_infobox)

{{Geobox
|name = नावासा आइलैंड
|image_caption = नवसा द्वीप [[अंतर्राष्ट्रीय अंतरिक्ष स्टेशन]] से देखा गया ]]
|country = {{झंडा। संयुक्त राज्य अमेरिका . आकार । =.. 23px}}
|location = [[कैरेबियन सागर]]
|area = 5.4
|length = 4.7
|width = 2.1
|image_name = नवासाइस14क्रॉपड.जपग
|pushpin_map = कैरिबियन
|pushpin_map_caption = लोकेशन इन दी कैरिबियन
|coordinates = {{कूर्ड|18|24|10|न|75|0|45|व|टाइप:आइसल_रीजिओन:उम|डिस्प्ले|=|इनलाइन,टाइटल}}
|area_km2 = 5.4
|length_km = 4.7
|width_km = 2.1
|native_name = 
}}
